In [ ]:
!pip install git+https://github.com/GerbenBeintema/deepSI@master
!pip install numpy, matplotlib, scipy

In [ ]:
import deepSI
import numpy as np
import cv2
from deepSI import System_data
from matplotlib import pyplot as plt

In [ ]:
# Load your data
filen = 'combined_data.npz'
data = np.load(filen)
frames = data['frames']  # Shape: (num_frames, height, width, channels)
forces_data = data['forces'][:, 3]  # Shape: (num_frames,)

# Assuming index 8300 is the start of your validation data
index = 8300

# Function to resize frames
def resize_frames_batch(frames, batch_size, new_height, new_width):
    num_frames = frames.shape[0]
    resized_frames = np.zeros((num_frames, new_height, new_width, frames.shape[3]), dtype=np.uint8)
    for start in range(0, num_frames, batch_size):
        end = start + batch_size
        batch_frames = frames[start:end]
        for i in range(batch_frames.shape[0]):
            resized_frames[start + i] = cv2.resize(batch_frames[i], (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    return resized_frames

# Resize parameters
new_height = frames.shape[1] // 8
new_width = frames.shape[2] // 8

batch_size = 100  # Adjust based on your system's capability
frames_resized = resize_frames_batch(frames, batch_size, new_height, new_width)
frames_train = frames_resized[:index]  # Training frames resized
frames_val = frames_resized[index:]  # Validation frames resized

# Reshape frames to match the expected input shape
frames_reshaped = frames_resized.transpose(0, 3, 1, 2)  # New shape: (num_frames, channels, height, width)
frames_val_reshaped = frames_val.transpose(0, 3, 1, 2)

# Split forces data into training and validation sets
forces_data_train = forces_data[:index]  # Training forces data
forces_data_val = forces_data[index:]  # Validation forces data

# Initialize the SS_encoder_CNN_video system
sys = deepSI.fit_systems.SS_encoder_CNN_video()

# Update shape based on resized data
n_channels, height, width = frames_reshaped.shape[1], frames_reshaped.shape[2], frames_reshaped.shape[3]
sys.init_nets(nu=1, ny=(n_channels, height, width))

# Create System_data instances with resized frames
system_data = System_data(u=forces_data_train, y=frames_reshaped[:index])
system_data_val = System_data(u=forces_data_val, y=frames_val_reshaped)

# Check shapes and types
print("Reshaped frames shape:", frames_reshaped.shape)
print("Forces data shape:", forces_data.shape)
print("Reshaped frames dtype:", frames_reshaped.dtype)
print("Forces data dtype:", forces_data.dtype)
print(system_data)

# Plot force data
plt.plot(system_data.u)
plt.xlabel('Time')
plt.ylabel('Force Data')
plt.title('Force Data Over Time')
plt.show()

# Fit the model
sys.fit(system_data, val_sys_data=system_data_val, concurrent_val=True, cuda=True)